In [ ]:
import sys 

stdoutOrigin=sys.stdout 
sys.stdout = open("C:/Users/tsb7592/Box/ARMADA Operations/Data and Analysis/poster.txt", "w")

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import *
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from scipy import interp
from sklearn.metrics import roc_auc_score

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# Load the Data

In [9]:
armada= pd.read_csv("C:/Users/tsb7592/Box/ARMADA Operations/Data and Analysis/May 2020 Interim Analysis Data for Manuscripts/ARMADA_TBSCORES_MAY2020.csv")
armada.shape

(1083, 189)

In [10]:
armada.assmnt.value_counts()

1    817
2    266
Name: assmnt, dtype: int64

In [11]:
cohort ={1:"Gen population healthy aging 85+", 2:"Gen population 65-85", 3:"Gen population MCI",
4:"Gen population AD", 5:"African American mild cognitive impairment",
6:"African American normal controls 65-85", 7:"Spanish Speaking mild cognitive impairment",
8:"Spanish Speaking normal controls 65-85", 9:"Spanish Speaking AD"}

In [12]:
#Rename the value under column cohort2
armada['cohort2'] = armada['cohort2'].map(cohort)

In [13]:
armada.cohort2.value_counts()

Gen population 65-85                          266
Gen population MCI                            140
Gen population healthy aging 85+              138
Spanish Speaking normal controls 65-85        130
African American normal controls 65-85        128
Spanish Speaking mild cognitive impairment    125
Gen population AD                              85
African American mild cognitive impairment     46
Spanish Speaking AD                            25
Name: cohort2, dtype: int64

In [14]:
#filter by assmnt = 1
armada = armada[armada.assmnt==1]

In [15]:
edu={1:"None",
2 :"Less than HS",
3 :"Less than HS",
4 :"Less than HS",
5 :"Less than HS",
6 :"Less than HS",
7 :"Less than HS",
8 :"Less than HS",
9 :"Less than HS",
10 :"Less than HS",
11 :"Less than HS",
12 :"Less than HS",
13 :"Less than HS",
14 :"Less than HS",
16 :"High School",
18 :"Some College",
20 :"Some College",
21 :"College Degree",
22 :"Graduate Degree",
23 :"Graduate Degree",
24 :"Graduate Degree",
25 :"Some College",
26 :"Some College",
27 :"Some College",
28 :"None",
999:"Unknown"}

# Poster Updates

In [16]:
select = [ 'Gen population 65-85', 'Gen population MCI','Gen population AD']

In [17]:
genpop = armada[armada.cohort2.isin(select)]

In [18]:
genpop['EDUCATION'] = genpop['EDUCATION'].map(edu)

In [19]:
co = {'Gen population 65-85':'3_Gen population 65-85', 'Gen population MCI': '2_Gen population MCI','Gen population AD': '1_Gen population AD'}

In [20]:
genpop['cohort'] = genpop['cohort2'].map(co)

In [21]:
co2 = {'Gen population 65-85':'1_Gen population 65-85', 'Gen population MCI': '2_Gen population MCI','Gen population AD': '3_Gen population AD'}

In [22]:
genpop['cohort2'] = genpop['cohort2'].map(co2)

In [23]:
select=[ '1_Gen population 65-85','2_Gen population MCI', '3_Gen population AD']

In [24]:
print(genpop.groupby('cohort2')['AGE'].agg(['mean', 'min', 'max']))
print('')

                        mean  min  max
cohort2                               
1_Gen population 65-85 72.72   65   84
2_Gen population MCI   76.85   64   99
3_Gen population AD    75.46   62   92



In [25]:
print(genpop.cohort2.value_counts())
print('')

1_Gen population 65-85    161
2_Gen population MCI      104
3_Gen population AD        71
Name: cohort2, dtype: int64



In [26]:
genpop.groupby('cohort2')['gender_rc'].value_counts(normalize=True)

cohort2                 gender_rc
1_Gen population 65-85  2           0.67
                        1           0.33
2_Gen population MCI    1           0.59
                        2           0.41
3_Gen population AD     1           0.58
                        2           0.42
Name: gender_rc, dtype: float64

In [27]:
#female =2
print(genpop.groupby('cohort2')['gender_rc'].value_counts().unstack())
print(genpop.groupby('cohort2')['gender_rc'].value_counts(normalize=True).unstack())
print('')
#2=hispanic
print(genpop.groupby('cohort2')['ethnicity_rc'].value_counts().unstack())
print(genpop.groupby('cohort2')['ethnicity_rc'].value_counts(normalize=True).unstack())
print('')
print(genpop.groupby('cohort2')['racetb_w'].value_counts().unstack())
print(genpop.groupby('cohort2')['racetb_w'].value_counts(normalize=True).unstack())
print('')
print(genpop.groupby('cohort2')['EDUCATION'].value_counts().unstack().reset_index())
print(genpop.groupby('cohort2')['EDUCATION'].value_counts(normalize=True).unstack().reset_index())
print('')
#1=Right
print(genpop.groupby('cohort2')['HANDEDNESS'].value_counts().unstack())
print(genpop.groupby('cohort2')['HANDEDNESS'].value_counts(normalize=True).unstack())
print('')

gender_rc                1    2
cohort2                        
1_Gen population 65-85  53  108
2_Gen population MCI    61   43
3_Gen population AD     41   30
gender_rc                 1    2
cohort2                         
1_Gen population 65-85 0.33 0.67
2_Gen population MCI   0.59 0.41
3_Gen population AD    0.58 0.42

ethnicity_rc              1  2
cohort2                       
1_Gen population 65-85  157  4
2_Gen population MCI    100  4
3_Gen population AD      68  3
ethnicity_rc              1    2
cohort2                         
1_Gen population 65-85 0.98 0.02
2_Gen population MCI   0.96 0.04
3_Gen population AD    0.96 0.04

racetb_w                 0    1
cohort2                        
1_Gen population 65-85  28  133
2_Gen population MCI    16   88
3_Gen population AD      7   64
racetb_w                  0    1
cohort2                         
1_Gen population 65-85 0.17 0.83
2_Gen population MCI   0.15 0.85
3_Gen population AD    0.10 0.90

EDUCATION                 c

In [28]:
cog = ['CCC_FCTScore','CFC_FCTScore', 'CTC_FCTScore', 'DCCS_FCTScore', 'Flanker_FCTScore', 'LSWM_FCTScore', 'ORR_FCTScore', 'PCPS_FCTScore', 'PSM_FCTScore', 'TPVT_FCTScore']

In [29]:
motor = ['Balance_FCTScore', 'Grip_fctd','Grip_fctnd','Peg9h_fctd','Peg9h_fctnd','Walk2Min_FCTScore', 'Walk4M_CScore']

In [30]:
sen =['Odor_FCTScore','VisualAc_FCTScore','WIN_Raw_L','WIN_Raw_R','PainInt_TScore']

In [31]:
emotion =['AngerAff_TScore','AngerHost_TScore', 'AngerPhysAg_TScore','EmoSup_TScore','FearAff_TScore','FearSom_TScore','Friend_TScore','GenLS_TScore','InstSup_TScore','Lone_TScore','MeanP_TScore','PHost_TScore','PosAff_TScore','PReject_TScore','PStress_TScore','Sad_TScore','SelfEff_TScore']

In [32]:
genpop.groupby('cohort2')[cog].count().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0)

cohort2,1_Gen population 65-85,2_Gen population MCI,3_Gen population AD
level_0,,,
CCC_FCTScore,159,87,63
CFC_FCTScore,159,79,41
CTC_FCTScore,159,79,41
DCCS_FCTScore,159,87,63
Flanker_FCTScore,159,87,62
LSWM_FCTScore,159,86,51
ORR_FCTScore,159,87,63
PCPS_FCTScore,159,87,64
PSM_FCTScore,159,79,46


In [33]:
print(genpop.groupby('cohort2')[cog].mean().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))

cohort2           1_Gen population 65-85  2_Gen population MCI  \
level_0                                                          
CCC_FCTScore                       58.20                 52.69   
CFC_FCTScore                       51.22                 41.49   
CTC_FCTScore                       55.54                 46.78   
DCCS_FCTScore                      55.16                 49.53   
Flanker_FCTScore                   45.56                 41.98   
LSWM_FCTScore                      53.65                 44.53   
ORR_FCTScore                       55.38                 51.93   
PCPS_FCTScore                      48.13                 42.24   
PSM_FCTScore                       52.06                 42.43   
TPVT_FCTScore                      59.52                 53.06   

cohort2           3_Gen population AD  
level_0                                
CCC_FCTScore                    49.73  
CFC_FCTScore                    30.39  
CTC_FCTScore                    38.46  
DCCS_FC

In [34]:
print(genpop.groupby('cohort2')[cog].std().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))
print('')                                                                  

cohort2           1_Gen population 65-85  2_Gen population MCI  \
level_0                                                          
CCC_FCTScore                        6.64                  7.31   
CFC_FCTScore                        8.87                  9.55   
CTC_FCTScore                        6.87                  7.72   
DCCS_FCTScore                      10.16                  9.36   
Flanker_FCTScore                    6.63                  6.93   
LSWM_FCTScore                       8.17                  9.65   
ORR_FCTScore                        6.53                  7.61   
PCPS_FCTScore                      12.55                 12.06   
PSM_FCTScore                       10.74                  8.63   
TPVT_FCTScore                       7.10                  7.84   

cohort2           3_Gen population AD  
level_0                                
CCC_FCTScore                     9.78  
CFC_FCTScore                    10.95  
CTC_FCTScore                    10.15  
DCCS_FC

In [35]:
print(genpop.groupby('cohort2')[motor].mean().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))

cohort2            1_Gen population 65-85  2_Gen population MCI  \
level_0                                                           
Balance_FCTScore                    49.54                 48.51   
Grip_fctd                           52.48                 49.92   
Grip_fctnd                          52.19                 49.79   
Peg9h_fctd                          48.57                 46.58   
Peg9h_fctnd                         49.40                 46.70   
Walk2Min_FCTScore                   49.70                 48.05   
Walk4M_CScore                        1.14                  1.10   

cohort2            3_Gen population AD  
level_0                                 
Balance_FCTScore                 47.28  
Grip_fctd                        48.48  
Grip_fctnd                       48.27  
Peg9h_fctd                       41.49  
Peg9h_fctnd                      41.48  
Walk2Min_FCTScore                40.48  
Walk4M_CScore                     1.01  


In [36]:
print(genpop.groupby('cohort2')[motor].std().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))
print("")

cohort2            1_Gen population 65-85  2_Gen population MCI  \
level_0                                                           
Balance_FCTScore                     9.58                 11.12   
Grip_fctd                            9.22                 10.12   
Grip_fctnd                           8.57                  9.97   
Peg9h_fctd                           9.09                  9.64   
Peg9h_fctnd                          9.57                  8.61   
Walk2Min_FCTScore                    8.77                  8.09   
Walk4M_CScore                        0.25                  0.25   

cohort2            3_Gen population AD  
level_0                                 
Balance_FCTScore                  9.52  
Grip_fctd                        10.07  
Grip_fctnd                        9.87  
Peg9h_fctd                       11.74  
Peg9h_fctnd                       9.38  
Walk2Min_FCTScore                10.32  
Walk4M_CScore                     0.25  



In [37]:
print(genpop.groupby('cohort2')[sen].mean().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))

cohort2            1_Gen population 65-85  2_Gen population MCI  \
level_0                                                           
Odor_FCTScore                       50.83                 43.40   
PainInt_TScore                      49.97                 50.56   
VisualAc_FCTScore                   52.56                 53.15   
WIN_Raw_L                           17.36                 12.95   
WIN_Raw_R                           18.56                 14.76   

cohort2            3_Gen population AD  
level_0                                 
Odor_FCTScore                    36.77  
PainInt_TScore                   49.04  
VisualAc_FCTScore                54.43  
WIN_Raw_L                        13.64  
WIN_Raw_R                        14.67  


In [38]:
print(genpop.groupby('cohort2')[sen].std().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))
print('')                                                          

cohort2            1_Gen population 65-85  2_Gen population MCI  \
level_0                                                           
Odor_FCTScore                       10.56                 11.05   
PainInt_TScore                       8.18                  8.72   
VisualAc_FCTScore                    8.98                 10.04   
WIN_Raw_L                            6.13                  7.72   
WIN_Raw_R                            6.02                  7.53   

cohort2            3_Gen population AD  
level_0                                 
Odor_FCTScore                     8.67  
PainInt_TScore                    6.88  
VisualAc_FCTScore                10.20  
WIN_Raw_L                         7.95  
WIN_Raw_R                         8.31  



In [39]:
print(genpop.groupby('cohort2')[emotion].mean().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))

cohort2             1_Gen population 65-85  2_Gen population MCI  \
level_0                                                            
AngerAff_TScore                      47.16                 46.96   
AngerHost_TScore                     42.69                 44.58   
AngerPhysAg_TScore                   44.61                 45.17   
EmoSup_TScore                        47.80                 46.84   
FearAff_TScore                       49.26                 50.49   
FearSom_TScore                       47.48                 47.37   
Friend_TScore                        51.02                 50.46   
GenLS_TScore                         56.35                 53.76   
InstSup_TScore                       49.80                 51.40   
Lone_TScore                          49.84                 49.61   
MeanP_TScore                         50.26                 49.04   
PHost_TScore                         48.89                 49.25   
PReject_TScore                       47.88      

In [40]:
print(genpop.groupby('cohort2')[emotion].std().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0))
print('')

cohort2             1_Gen population 65-85  2_Gen population MCI  \
level_0                                                            
AngerAff_TScore                       8.30                  9.58   
AngerHost_TScore                      7.49                  8.59   
AngerPhysAg_TScore                    5.16                  6.67   
EmoSup_TScore                         8.51                  8.21   
FearAff_TScore                        8.76                  9.72   
FearSom_TScore                        7.91                  9.29   
Friend_TScore                         9.69                  8.28   
GenLS_TScore                          8.86                  9.30   
InstSup_TScore                       10.37                  9.26   
Lone_TScore                           9.18                  9.35   
MeanP_TScore                          8.76                  8.95   
PHost_TScore                          7.79                  8.25   
PReject_TScore                        8.59      

In [41]:
sys.stdout.close()
sys.stdout=stdoutOrigin

NameError: name 'sys' is not defined

In [42]:
# calculate the Cohen's d between two samples
from numpy.random import randn
from numpy.random import seed
from numpy import mean
from numpy import var
from math import sqrt
 
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)
    # calculate the effect size
    return (u1 - u2) / s


In [43]:
def tukey_test(df, var):
    df =df[['cohort', var]].dropna()
    tukey = pairwise_tukeyhsd(endog=df[var],
                          groups=df['cohort'],
                          alpha=0.05)

    return(tukey)
    

In [44]:
def get_list(li, element):
    list_of_numbers = []
    list_of_numbers.append(element)
    for tup in li:
        try:
            list_of_numbers.append(round(float(tup),2))
        except ValueError:
            pass
        
    del list_of_numbers[2]
    
    CI = '['+str(list_of_numbers[2]) + ', ' + str(list_of_numbers[3])+']'
    list_of_numbers = list_of_numbers[:2]
    list_of_numbers.append(CI)
    list_of_numbers.append(li[-1])
    
    
    
    return list_of_numbers

In [51]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def anova_one_var(data, var):
    #Test age
    li =[]
    for cohort in select:
        
        li.append(data[data.cohort2== cohort][var].dropna())

    F, p =f_oneway(li[0], li[1],li[2])
    N = len(li[0]+li[1]+li[2])
    print(var, ': degree of freedom', N-3)
    print('F statistic = {:5.2f} and probability p = {:5.2f}'.format(F, p))
    
    # calculate cohen's d
    d1 = round(cohend(li[0], li[1]),2)
    #print('Cohens d: %.2f (' % d1, select[0],'vs ',select[1], ")")
    d2 = round(cohend(li[0], li[2]),2)
    #print('Cohens d: %.2f (' % d2,select[0],'vs ',select[2],')')
    d3 = round(cohend(li[1], li[2]),2)
    #print('Cohens d: %.2f (' % d3, select[1],'vs ',select[2],")")
    coh = [[d3, d2, d1]]
    
    rdf = pd.DataFrame(coh, columns =['AD vs. MCI','AD vs. 65-85','MCI vs 65-85'], dtype = float)
    
    
    if p <= 0.05:
        rv = tukey_test(data, var)
        rv = str(rv) 
        
        l1 = rv.split('\n')[4].split(' ')
        l2 = rv.split('\n')[5].split(' ')
        l3 = rv.split('\n')[6].split(' ')
        
        a1 =get_list(l1, d3)
        a2 =get_list(l2, d2)
        a3 =get_list(l3, d1)
        print("")
        
        rdf = pd.DataFrame(list(zip(a1,a2, a3)), index =[ "Cohen's d", 'T-value diff', '95% CI', 'Tukey p<0.05'],
                                              columns =['AD vs. MCI','AD vs. 65-85','MCI vs 65-85'])
        
   
    display(rdf)
    #return rdf
    print('')

In [46]:

from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
#doc = SimpleDocTemplate("C:/Users/tsb7592/Downloads/simple_table.pdf", pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)



In [52]:

for col in cog:
    data = anova_one_var( genpop, col)
    #t=Table(data,5*[0.4*inch], 4*[0.4*inch])
    #elements.append(t)
    

# write the document to disk
#doc.build(elements)

CCC_FCTScore : degree of freedom 306
F statistic = 33.59 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.35,1.11,0.80
T-value diff,2.96,8.47,5.51
95% CI,"[0.01, 5.91]","[5.82, 11.12]","[3.14, 7.89]"
Tukey p<0.05,True,True,True



CFC_FCTScore : degree of freedom 276
F statistic = 89.72 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,1.11,2.23,1.07
T-value diff,11.10,20.83,9.73
95% CI,"[6.84, 15.36]","[16.95, 24.71]","[6.68, 12.77]"
Tukey p<0.05,True,True,True



CTC_FCTScore : degree of freedom 276
F statistic = 94.11 and probability p =  0.00



...

,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.90,1.35,0.48
T-value diff,11.44,17.33,5.88
95% CI,"[6.54, 16.35]","[12.92, 21.73]","[1.92, 9.85]"
Tukey p<0.05,True,True,True



PSM_FCTScore : degree of freedom 281
F statistic = 61.40 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.82,1.62,0.95
T-value diff,6.39,16.01,9.63
95% CI,"[2.21, 10.57]","[12.24, 19.78]","[6.53, 12.73]"
Tukey p<0.05,True,True,True



TPVT_FCTScore : degree of freedom 307
F statistic = 35.45 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.29,1.10,0.88
T-value diff,2.70,9.16,6.46
95% CI,"[-0.48, 5.87]","[6.31, 12.02]","[3.89, 9.04]"
Tukey p<0.05,False,True,True


In [53]:
#The one-way ANOVA tests the null hypothesis that 3 genpop groups have the same population mean for motor vars
for col in motor:
    anova_one_var(genpop, col)


Balance_FCTScore : degree of freedom 240
F statistic =  0.93 and probability p =  0.40


,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,0.12,0.24,0.10



Grip_fctd : degree of freedom 306
F statistic =  4.57 and probability p =  0.01



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.14,0.42,0.27
T-value diff,1.43,3.99,2.56
95% CI,"[-2.32, 5.19]","[0.63, 7.36]","[-0.48, 5.6]"
Tukey p<0.05,False,True,False



Grip_fctnd : degree of freedom 306
F statistic =  4.71 and probability p =  0.01



...

,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.58,0.83,0.29
T-value diff,5.23,7.93,2.70
95% CI,"[1.59, 8.87]","[4.67, 11.18]","[-0.25, 5.65]"
Tukey p<0.05,True,True,False



Walk2Min_FCTScore : degree of freedom 273
F statistic = 20.84 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.84,1.00,0.19
T-value diff,7.57,9.22,1.64
95% CI,"[3.79, 11.36]","[5.84, 12.6]","[-1.33, 4.62]"
Tukey p<0.05,True,True,False



Walk4M_CScore : degree of freedom 295
F statistic =  5.83 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.37,0.53,0.16
T-value diff,0.09,0.13,0.04
95% CI,"[-0.01, 0.19]","[0.04, 0.23]","[-0.04, 0.12]"
Tukey p<0.05,False,True,False


In [54]:
#The one-way ANOVA tests the null hypothesis that 3 genpop groups have the same population mean for sensation vars
for col in sen:
    print(col)
    anova_one_var(genpop, col)

Odor_FCTScore
Odor_FCTScore : degree of freedom 302
F statistic = 44.20 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.65,1.40,0.69
T-value diff,6.63,14.06,7.43
95% CI,"[2.54, 10.72]","[10.39, 17.73]","[4.15, 10.71]"
Tukey p<0.05,True,True,True



VisualAc_FCTScore
VisualAc_FCTScore : degree of freedom 305
F statistic =  0.87 and probability p =  0.42


,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,-0.13,-0.20,-0.06



WIN_Raw_L
WIN_Raw_L : degree of freedom 324
F statistic = 14.34 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,-0.09,0.56,0.65
T-value diff,-0.69,3.73,4.41
95% CI,"[-3.31, 1.94]","[1.3, 6.15]","[2.31, 6.52]"
Tukey p<0.05,False,True,True



WIN_Raw_R
WIN_Raw_R : degree of freedom 324
F statistic = 12.26 and probability p =  0.00



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.01,0.58,0.57
T-value diff,0.10,3.90,3.80
95% CI,"[-2.52, 2.71]","[1.48, 6.31]","[1.7, 5.9]"
Tukey p<0.05,False,True,True



PainInt_TScore
PainInt_TScore : degree of freedom 332
F statistic =  0.74 and probability p =  0.48


,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,0.19,0.12,-0.07


In [55]:
#The one-way ANOVA tests the null hypothesis that 3 genpop groups have the same population mean for emotion vars
for col in emotion:
    print(col) 
    anova_one_var(genpop, col)


AngerAff_TScore
AngerAff_TScore : degree of freedom 331
F statistic =  0.62 and probability p =  0.54


,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,-0.15,-0.14,0.02



AngerHost_TScore
AngerHost_TScore : degree of freedom 331
F statistic =  1.80 and probability p =  0.17


,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,0.18,-0.05,-0.24



AngerPhysAg_TScore
AngerPhysAg_TScore : degree of freedom 331
F statistic =  3.30 and probability p =  0.04



...

,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,-0.11,-0.24,-0.12



Sad_TScore
Sad_TScore : degree of freedom 331
F statistic =  1.93 and probability p =  0.15


,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
0,-0.13,-0.27,-0.15



SelfEff_TScore
SelfEff_TScore : degree of freedom 331
F statistic =  4.21 and probability p =  0.02



,AD vs. MCI,AD vs. 65-85,MCI vs 65-85
Cohen's d,0.28,0.42,0.13
T-value diff,2.52,3.61,1.10
95% CI,"[-0.65, 5.68]","[0.68, 6.54]","[-1.47, 3.66]"
Tukey p<0.05,False,True,False


# Merging uds

In [66]:
pain = []
for col in armada.columns:
    if 'Pain' in col:
        pain.append(col)

In [67]:
#Data saved in csv/excel
import glob, os

data_dir= "C:/Users/tsb7592/Box/ARMADA Operations/Data and Analysis/Mar 2021 UDS data (deidentified)"
os.chdir(data_dir)

In [68]:
uds= pd.read_csv("ARMADA_UDS_MAR21_DI.CSV")
uds.shape

(1046, 618)

In [ ]:
#Combine ARMADA with this UDS left join
df = pd.merge(armada,uds, how="left", on=['ptid','assmnt'])

In [ ]:
#MEMORY through comport (so total of 9 variables)
keep = ['MEMORY', 'ORIENT', 'JUDGMENT', 'COMMUN', 'HOMEHOBB', 'PERSCARE',
       'CDRSUM', 'CDRGLOB', 'COMPORT']

In [ ]:
#Only keep the rows with a valid MINDATEDIFF
df =df[df.MINDATEDIFF.notnull()]

In [ ]:
genpop = df[df.cohort2.isin(select)]

# 1) Count for each variable across all 3 gen pop groups 

In [ ]:
counts  = genpop.groupby('cohort2')[keep].count().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0)

In [ ]:
counts =counts.reindex(keep)[select]
print(counts)
print("")

# 2) ANOVA & Tukey’s Test ('CDRSUM', 'CDRGLOB')

https://www.statology.org/tukey-test-python/
http://pytolearn.csd.auth.gr/d1-hyptest/12/anova-one.html

In [ ]:
genpop_melt = pd.melt(genpop[keep[6:8]].reset_index(), id_vars=['index'], value_vars=keep[6:8])
# replace column namesg
genpop_melt.columns = ['index', 'col', 'value']

In [ ]:
genpop_melt=genpop_melt.dropna()

In [ ]:
genpop_melt

In [ ]:
# perform Tukey's test
tukey = pairwise_tukeyhsd(endog=genpop_melt['value'],
                          groups=genpop_melt['col'],
                          alpha=0.05)

#display results
print(tukey)
print('')

# 3)  NACCFAM counts per group

In [ ]:
df.NACCFAM.value_counts()

In [ ]:
#Drop the unknown NACCFAM 
Ncounts  = genpop[genpop.NACCFAM !=9 ].groupby('NACCFAM')['cohort2'].value_counts().unstack()
#Ncounts.to_excel('C:/Users/tsb7592/Downloads/NACCFAM.xlsx')

In [ ]:
print(Ncounts)

In [ ]:
#Chi square results got from R

#Pearson's Chi-squared test
print('X-squared = 2.7786, df = 2, p-value = 0.2493')
print("")

# 4) HYPERT counts per group

In [ ]:
df.HYPERTEN.unique()

In [ ]:
df.HYPERCHO.value_counts()

In [ ]:
h1=genpop[(genpop.HYPERTEN != 9 )&(genpop.HYPERTEN !=-4)].groupby('HYPERTEN')['cohort2'].value_counts().unstack().fillna(0)
print(h1)

In [ ]:
#Pearson's Chi-squared test

print("X-squared = 10.077, df = 4, p-value = 0.03916")
print("")

In [ ]:
h2=genpop[(genpop.HYPERCHO != 9 )&(genpop.HYPERCHO !=-4)].groupby('HYPERCHO')['cohort2'].value_counts().unstack().fillna(0)
print(h2)

In [ ]:
#Pearson's Chi-squared test

#data:  h2
print("X-squared = 4.9095, df = 4, p-value = 0.2967")


# Armada Pain
For each of the four Odor variables:
Compare them by cohort (so there should be nine in total)

In [ ]:
Odor =[]
for val in armada.columns:
    if 'Odor' in val:
        Odor.append(val)
    if 'Pain' in val:
        print(val)

In [ ]:
armada.cohort2.unique()

In [ ]:
odor = armada.groupby('cohort2')[Odor].count().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0)
odor

# Histogram (9)
Boxplot (nine boxplots)
Table with Ns, Min, Median, Mean, Max, and SD
One row for all data
Nine rows for each of the cohorts
Table of 10 x 6

In [ ]:
#Data saved in csv/excel
import glob, os

data_dir= "C:/Users/tsb7592/Box/ARMADA Operations/Data and Analysis"
os.chdir(data_dir)

In [ ]:
Odor

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for var in Odor:
    sns.set() 
    plt.figure(figsize=(8, 6))
    g = sns.FacetGrid(armada, col="cohort2" ,hue="cohort2",size=4, aspect=1,col_wrap=3)
    g.fig.suptitle(var)
    g.map(sns.histplot, var)
    plt.savefig(str(var)+'_hist.png')

In [ ]:
 
for var in Odor:
    g = sns.factorplot('cohort2', var,  data=armada, kind="box",size=4, aspect=2)
    g.fig.suptitle(var)
    
    for axes in g.axes.flat:
        _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=30, horizontalalignment='right')
    
    plt.savefig(str(var)+'_boxplot.png',bbox_inches='tight')

In [ ]:
amelt = pd.melt(armada.reset_index(), id_vars=['index', 'cohort2'], value_vars=Odor)
# replace column namesg
amelt.columns = ['index','cohort2', 'col', 'value']

In [ ]:
amelt = amelt.dropna()

In [ ]:
ALL =amelt.groupby('cohort2')['value'].describe().iloc[:,[1,2]].round(2)

In [ ]:
ALL

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/tsb7592/Box/ARMADA Operations/Data and Analysis/odor_tables.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
odor.to_excel(writer, sheet_name='by_cohort')

for var in Odor:
    ta =armada.groupby('cohort2')[var].describe().unstack().reset_index().pivot(index='level_0', columns='cohort2', values=0)
    ta= ta.iloc[[3,6,1,5,7,4],:].round(2)
    ta.index =['N', 'min', 'median', 'mean', 'std', 'max']
    ta.to_excel(writer, sheet_name=var)
    display(ta)

ALL.to_excel(writer, sheet_name='all')   
# Close the Pandas Excel writer and output the Excel file.
writer.save()